# מנגנוני קשב ומודלים של טרנספורמרים

חיסרון מרכזי של רשתות חוזרות (RNN) הוא שכל המילים ברצף משפיעות באותה מידה על התוצאה. זה מוביל לביצועים תת-אופטימליים במודלים סטנדרטיים של LSTM עבור משימות רצף לרצף, כמו זיהוי ישויות בשם ותרגום מכונה. במציאות, למילים מסוימות ברצף הקלט יש השפעה רבה יותר על הפלט מאשר לאחרות.

נבחן מודל רצף לרצף, כמו תרגום מכונה. הוא מיושם באמצעות שתי רשתות חוזרות, כאשר רשת אחת (**מקודד**) דוחסת את רצף הקלט למצב מוסתר, ורשת אחרת, **מפענח**, פורסת את המצב המוסתר הזה לתוצאה מתורגמת. הבעיה בגישה זו היא שהמצב הסופי של הרשת מתקשה לזכור את תחילת המשפט, מה שמוביל לאיכות ירודה של המודל במשפטים ארוכים.

**מנגנוני קשב** מספקים דרך לשקלל את ההשפעה ההקשרית של כל וקטור קלט על כל תחזית פלט של ה-RNN. זה מיושם על ידי יצירת קיצורי דרך בין המצבים הביניים של ה-RNN של הקלט לבין ה-RNN של הפלט. כך, בעת יצירת סמל פלט $y_t$, ניקח בחשבון את כל המצבים המוסתרים של הקלט $h_i$, עם מקדמי משקל שונים $\alpha_{t,i}$.

![תמונה המציגה מודל מקודד/מפענח עם שכבת קשב אדיטיבית](../../../../../translated_images/he/encoder-decoder-attention.7a726296894fb567.webp)
*מודל מקודד-מפענח עם מנגנון קשב אדיטיבי מתוך [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), מצוטט מתוך [פוסט הבלוג הזה](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

מטריצת הקשב $\{\alpha_{i,j}\}$ מייצגת את המידה שבה מילים מסוימות בקלט משפיעות על יצירת מילה מסוימת ברצף הפלט. להלן דוגמה למטריצה כזו:

![תמונה המציגה יישור דוגמה שנמצא על ידי RNNsearch-50, מתוך Bahdanau - arviz.org](../../../../../translated_images/he/bahdanau-fig3.09ba2d37f202a6af.webp)

*תמונה מתוך [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (איור 3)*

מנגנוני קשב אחראים לחלק ניכר מהמצב הנוכחי או הקרוב למצב הנוכחי של האמנות בתחום עיבוד שפה טבעית. עם זאת, הוספת קשב מגדילה מאוד את מספר הפרמטרים במודל, מה שהוביל לבעיות קנה מידה עם RNNs. מגבלה מרכזית בקנה מידה של RNNs היא שהאופי החוזר של המודלים מקשה על ביצוע אצווה (batching) והקבלה במקביל. ב-RNN, כל אלמנט ברצף צריך להיות מעובד בסדר רציף, מה שאומר שלא ניתן להקביל בקלות את התהליך.

האימוץ של מנגנוני קשב בשילוב עם מגבלה זו הוביל ליצירת מודלים של טרנספורמרים, שהם כיום המצב המתקדם ביותר (State of the Art) שאנו מכירים ומשתמשים בהם כיום, כמו BERT ו-OpenGPT3.

## מודלים של טרנספורמרים

במקום להעביר את ההקשר של כל תחזית קודמת לשלב ההערכה הבא, **מודלים של טרנספורמרים** משתמשים ב**קידודים מיקום** ובקשב כדי ללכוד את ההקשר של קלט נתון בתוך חלון טקסט מסוים. התמונה למטה מראה כיצד קידודי מיקום עם קשב יכולים ללכוד הקשר בתוך חלון נתון.

![GIF מונפש המציג כיצד מתבצעות ההערכות במודלים של טרנספורמרים.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

מכיוון שכל מיקום קלט ממופה באופן עצמאי לכל מיקום פלט, טרנספורמרים יכולים להקביל טוב יותר מ-RNNs, מה שמאפשר מודלים לשוניים גדולים ומבטאים יותר. כל ראש קשב יכול לשמש ללמידת יחסים שונים בין מילים, מה שמשפר משימות עיבוד שפה טבעית.

**BERT** (ייצוגי מקודד דו-כיווניים מטרנספורמרים) הוא רשת טרנספורמרים גדולה מאוד עם 12 שכבות עבור *BERT-base* ו-24 עבור *BERT-large*. המודל מאומן מראש על מאגר טקסטים גדול (ויקיפדיה + ספרים) באמצעות אימון לא מפוקח (חיזוי מילים מוסתרות במשפט). במהלך האימון המוקדם, המודל סופג רמה משמעותית של הבנת שפה, שניתן לנצל לאחר מכן עם מערכי נתונים אחרים באמצעות כוונון עדין. תהליך זה נקרא **למידת העברה**.

![תמונה מתוך http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/he/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

ישנן וריאציות רבות של ארכיטקטורות טרנספורמרים, כולל BERT, DistilBERT, BigBird, OpenGPT3 ועוד, שניתן לכוונן. חבילת [HuggingFace](https://github.com/huggingface/) מספקת מאגר לאימון רבות מהארכיטקטורות הללו עם PyTorch.

## שימוש ב-BERT לסיווג טקסט

בואו נראה כיצד ניתן להשתמש במודל BERT מאומן מראש לפתרון המשימה המסורתית שלנו: סיווג רצפים. נסווג את מערך הנתונים המקורי שלנו, AG News.

ראשית, נטען את ספריית HuggingFace ואת מערך הנתונים שלנו:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


מכיוון שנשתמש במודל BERT שהוכשר מראש, נצטרך להשתמש בטוקנייזר ספציפי. ראשית, נטען טוקנייזר שמקושר למודל BERT שהוכשר מראש.

ספריית HuggingFace מכילה מאגר של מודלים שהוכשרו מראש, שניתן להשתמש בהם פשוט על ידי ציון שמם כארגומנט לפונקציות `from_pretrained`. כל הקבצים הבינאריים הנדרשים עבור המודל יירדו באופן אוטומטי.

עם זאת, במקרים מסוימים תצטרכו לטעון מודלים משלכם. במקרה כזה, תוכלו לציין את הספרייה שמכילה את כל הקבצים הרלוונטיים, כולל פרמטרים עבור הטוקנייזר, קובץ `config.json` עם פרמטרי המודל, משקלים בינאריים וכו'.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

האובייקט `tokenizer` מכיל את הפונקציה `encode` שניתן להשתמש בה ישירות לקידוד טקסט:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

לאחר מכן, בואו ניצור איטרטורים שנשתמש בהם במהלך האימון כדי לגשת לנתונים. מכיוון ש-BERT משתמש בפונקציית קידוד משלו, נצטרך להגדיר פונקציית ריפוד דומה ל-`padify` שהגדרנו קודם:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

במקרה שלנו, נשתמש במודל BERT מאומן מראש שנקרא `bert-base-uncased`. בואו נטען את המודל באמצעות חבילת `BertForSequenceClassfication`. זה מבטיח שלמודל שלנו כבר יש את הארכיטקטורה הנדרשת לסיווג, כולל המסווג הסופי. תראו הודעת אזהרה שמציינת שהמשקלים של המסווג הסופי לא מאותחלים, והמודל ידרוש אימון מוקדם - זה בסדר גמור, כי זה בדיוק מה שאנחנו עומדים לעשות!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

עכשיו אנחנו מוכנים להתחיל באימון! מכיוון ש-BERT כבר עבר אימון מוקדם, אנחנו רוצים להתחיל עם קצב למידה קטן יחסית כדי לא להרוס את המשקלים ההתחלתיים.

כל העבודה הקשה מתבצעת על ידי המודל `BertForSequenceClassification`. כאשר אנחנו מפעילים את המודל על נתוני האימון, הוא מחזיר גם את ההפסד (loss) וגם את הפלט של הרשת עבור המיני-באטץ' שהוזן. אנחנו משתמשים בהפסד לאופטימיזציה של הפרמטרים (`loss.backward()` מבצע את המעבר לאחור), וב-`out` לחישוב דיוק האימון על ידי השוואת התוויות שהתקבלו `labs` (מחושבות באמצעות `argmax`) עם התוויות הצפויות `labels`.

כדי לשלוט בתהליך, אנחנו מצטברים את ההפסד והדיוק לאורך כמה איטרציות, ומדפיסים אותם כל `report_freq` מחזורי אימון.

האימון הזה כנראה ייקח זמן רב, ולכן אנחנו מגבילים את מספר האיטרציות.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


אתם יכולים לראות (במיוחד אם תגדילו את מספר האיטרציות ותמתינו מספיק זמן) שסיווג באמצעות BERT נותן לנו דיוק די טוב! זאת מכיוון ש-BERT כבר מבין היטב את מבנה השפה, ואנחנו רק צריכים לכוונן את המסווג הסופי. עם זאת, מכיוון ש-BERT הוא מודל גדול, תהליך האימון כולו לוקח זמן רב ודורש כוח חישוב משמעותי! (GPU, ועדיף יותר מאחד).

> **Note:** בדוגמה שלנו, השתמשנו באחד מהמודלים הקטנים ביותר של BERT שהוכנו מראש. ישנם מודלים גדולים יותר שסביר להניח שיניבו תוצאות טובות יותר.


## הערכת ביצועי המודל

עכשיו נוכל להעריך את ביצועי המודל שלנו על קבוצת הנתונים לבדיקות. לולאת ההערכה דומה מאוד ללולאת האימון, אבל חשוב לא לשכוח להעביר את המודל למצב הערכה על ידי קריאה ל-`model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## נקודות חשובות

ביחידה זו ראינו כמה קל לקחת מודל שפה מאומן מראש מספריית **transformers** ולהתאים אותו למשימת סיווג הטקסט שלנו. באופן דומה, ניתן להשתמש במודלים של BERT לחילוץ ישויות, מענה על שאלות ומשימות NLP אחרות.

מודלים של Transformer מייצגים את המצב המתקדם ביותר כיום בתחום ה-NLP, וברוב המקרים הם צריכים להיות הפתרון הראשון שבו תתחילו להתנסות כאשר אתם מיישמים פתרונות NLP מותאמים אישית. עם זאת, הבנת העקרונות הבסיסיים של רשתות עצביות חוזרות שנדונו במודול זה היא חשובה מאוד אם אתם רוצים לבנות מודלים עצביים מתקדמים.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור הסמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
